### Data Loading and Label Mapping

In [ ]:
#IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
from googlesearch import search
from bs4 import BeautifulSoup
import requests
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
from scraping_selenium import people_also_ask
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [ ]:
df = pd.read_csv('Test_dataset(FINAL).csv')
df

In [ ]:
# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
df['Label'].value_counts()

In [ ]:
df['Label'].value_counts().plot(kind='barh')

Article Pipeline Utility

In [ ]:

def get_top_links(query, num_links=1):
    linksgot = []
    try:
        # Perform Google search and get the top links
        search_results = search(query, num_results=num_links)

        # Print the top links
        for i, link in enumerate(search_results, start=1):
            linksgot.append(link)
    except Exception as e:
        print(f"An error occurred: {e}")
    return linksgot


def get_title_and_content(search_query_results):
    article_titles = []
    article_content = []

    # Set up Selenium options (headless mode if needed)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument('log-level=1')

    cService = Service(executable_path='PATH_TO_SELENIUM_WEBDRIVER')
    driver = webdriver.Chrome(service=cService,options=chrome_options)

    if search_query_results:
        for result in search_query_results:
            try:
                # Navigate to the URL
                driver.get(result)

                # Scrape <h1> tags for titles
                h1_tags = driver.find_elements(By.TAG_NAME, 'h1')
                currenth1 = " ".join([h1.text for h1 in h1_tags])
                article_titles.append(currenth1)

                # Scrape <p> tags for content
                p_tags = driver.find_elements(By.TAG_NAME, 'p')
                currentp = " ".join([p.text for p in p_tags])
                article_content.append(currentp)

            except Exception as e:
                print(f"An error occurred: {e}")

    # Quit the driver'
    driver.close()
    driver.quit()

    return article_titles, article_content


# Get the titles and contents
def make_data(search_query_results):
    titles, contents = get_title_and_content(
        get_top_links(search_query_results))

    # Create a pandas DataFrame
    data = {'Title': titles, 'Content': contents}
    df = pd.DataFrame(data)
    return df

### SLM Pipeline

In [ ]:
#Load FactCC Model pipeline
pipe = pipeline(model="manueldeprada/FactCC", task="text-classification", max_length=512)

In [ ]:
#To answer the question generated by SLM via google
def google_search(query):
    headers = {
        'User-agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'
    }
    
    # Perform the Google search
    query = re.sub(r'[^a-zA-Z0-9.,!?;:()\- ]+', '', query)
    search_url = f'https://www.google.com/search?q={query}'
    html = requests.get(search_url, headers=headers)
    
    # Parse the HTML response
    soup = BeautifulSoup(html.text, 'html.parser')
    
    #extract answer from summary answer on google
    if soup.select_one('.hgKElc'):
        answer = soup.select_one('.hgKElc').text 

    # Extract the answer (assuming it's in a specific class) -> QUICK ANSWER BOX
    elif soup.select_one('.DI6Ufb'):
        answer = soup.select_one('.DI6Ufb')
        answer = answer.find(class_='Z0LcW t2b5Cf').text
        answer=query+' '+answer
    
    elif not soup.select_one('.DI6Ufb'):
        result=people_also_ask(search_url)
        # print("Result of the Scraping",result)
        if len(result)==2:
            answer=f'{result[0]} {result[1]}'
        else:
            #incase people also asked also comes empty we revert to the article pipeline
            scraped_df = make_data(query)
            scraped_df.dropna(inplace=True)

            # Convert the 'Content' column to strings
            scraped_df['Content'] = scraped_df['Content'].astype(str)

            # Sort the DataFrame based on the length of the strings in the 'Content' column
            scraped_df = scraped_df.sort_values(by='Content', key=lambda x: x.str.len(), ascending=False)

            if len(scraped_df) == 0 or not scraped_df['Content'][0] or '403 Forbidden' in scraped_df['Content'][0] or '403 Forbidden' in scraped_df['Title'][0] :
                #Could not retrieve articles related to headline, Could possibly be a false claim OR Scraper got blocked/forbidden
                return str('Cannot retrieve articles, most likely to be false')

            scraped_content = (
            f"{scraped_df['Title'][0]} \n{scraped_df['Content'][0]}")
            answer = scraped_content
    return answer

In [ ]:
def FactCC_Phi(input_headline ,Phi_generated_question):
    quickSearchAnswer = google_search(Phi_generated_question)

    #compare scraped info as source and headline as the claim
    ans = pipe([[[quickSearchAnswer,Phi_generated_question]]], truncation=True, padding='max_length')

    print('Input headline : ', input_headline)
    print('Phi-3 Generated Question : ', Phi_generated_question)
    print('Scraped Answer : ', quickSearchAnswer)
    if ans[0]['label'] == 'CORRECT':
        return True,quickSearchAnswer
    else:
        return False,quickSearchAnswer

In [ ]:
model_decision,scraped_content = FactCC_Phi(df['Headline'][0],df['Question_phi'][0])
actual_decision = df['Label'][0]
print(model_decision,actual_decision)
if (model_decision) == (actual_decision):
    print('Correct Decision')
else:
    print('Incorrect Decision')

In [ ]:
df

In [ ]:
import csv
import os
from tqdm import tqdm
predictions = []
actuals = []
results = []
batch_size = 2
csv_filename = 'Pipeline_SLM(Phi).csv'
processed_headlines = set()

# Check if CSV file already exists and load processed headlines
if os.path.exists(csv_filename):
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='|')
        next(csvreader)  # Skip header
        for row in csvreader:
            if len(row) < 2:  # Skip rows that may not have all columns
                continue
            headline = row[1]
            headline = headline.str.strip("'\"")
            headline = headline.lower()
            processed_headlines.add(headline)


try:
    # Open the CSV file for writing with '|' as the delimiter
    with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='|')
        if not processed_headlines:
            csvwriter.writerow(['Index', 'Headline','Scraped_Content','Question_phi', 'Model Decision', 'Actual Decision'])
        
        # Iterate over each row in the dataframe
        for index, row in tqdm(df.iterrows(),total= len(df)):
            try:
                headline = row['Headline']
                if headline in processed_headlines:
                    print(f"Skipping already processed headline: {headline}")
                    continue
                model_decision,scraped_content = FactCC_Phi(row['Headline'], row['Question_phi'])
                # Convert scraped content to a single line
                scraped_content_single_line = ' '.join(scraped_content.splitlines()).strip().replace('|', ' ')
                actual_decision = row['Label']
                predictions.append(model_decision)
                actuals.append(actual_decision)
                
                # Write the results to the CSV file
                results.append([index, row['Headline'], scraped_content_single_line, row['Question_phi'],model_decision, actual_decision])
                
                # print(f"Index: {index}, Model Decision: {model_decision}, Actual Decision: {actual_decision}")
                # print(f"------------------------------------------------------------------------------------")
                time.sleep(5) 
                if len(results) % batch_size == 0:
                        csvwriter.writerows(results)
                        csvfile.flush()
                        os.fsync(csvfile.fileno())
                        results = [] # Delay to prevent rate limiting
            except Exception as e:
                print(f"Error processing row {index}: {str(e)}")
                continue

        # Write any remaining results
        if results:
            csvwriter.writerows(results)
            csvfile.flush()
            os.fsync(csvfile.fileno())

    # Calculate the metrics
    accuracy = accuracy_score(actuals, predictions)
    precision = precision_score(actuals, predictions, pos_label=True)
    recall = recall_score(actuals, predictions, pos_label=True)
    f1 = f1_score(actuals, predictions, pos_label=True)

    # Append the metrics to the CSV file
    with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='|')
        csvwriter.writerow([])  # Add an empty row for separation
        csvwriter.writerow(['Metric', 'Value'])
        csvwriter.writerow(['Accuracy', accuracy])
        csvwriter.writerow(['Precision', precision])
        csvwriter.writerow(['Recall', recall])
        csvwriter.writerow(['F1 Score', f1])

    print(f'Results written to {csv_filename}')
    print(f'Final file size: {os.path.getsize(csv_filename)} bytes')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    if os.path.exists(csv_filename):
        print(f"CSV file exists. Size: {os.path.getsize(csv_filename)} bytes")
    else:
        print("CSV file does not exist.")